print("Hello world")

print("Hello ")

In [1]:
!mkdir -p ~/.kaggle


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:51<00:00, 22.3MB/s]
100% 1.06G/1.06G [00:51<00:00, 22.0MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
#generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)


validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 20000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [ ]:
#normalization
def process(image,label):
  image = tf.cast(image/225,tf.float32)
  return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)


In [ ]:
#create CNN Model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding = 'valid',activation='relu',input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2,padding = 'valid'))

model.add(Conv2D(68,kernel_size=(3,3),padding = 'valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2,padding = 'valid'))


model.add(Conv2D(128,kernel_size=(3,3),padding = 'valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2,padding = 'valid'))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 68)        │          19,652 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 68)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          78,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,853,061 (56.66 MB)

 Trainable params: 14,853,061 (56.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs = 10,validation_data=validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 89s 128ms/step - accuracy: 0.5807 - loss: 0.6765 - val_accuracy: 0.7416 - val_loss: 0.5325
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 116ms/step - accuracy: 0.7512 - loss: 0.5106 - val_accuracy: 0.7933 - val_loss: 0.4302
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 85s 136ms/step - accuracy: 0.8238 - loss: 0.3883 - val_accuracy: 0.8683 - val_loss: 0.3123
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 130s 116ms/step - accuracy: 0.8942 - loss: 0.2551 - val_accuracy: 0.8996 - val_loss: 0.2786
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 85s 137ms/step - accuracy: 0.9467 - loss: 0.1401 - val_accuracy: 0.9297 - val_loss: 0.2139
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 116ms/step - accuracy: 0.9687 - loss: 0.0850 - val_accuracy: 0.9336 - val_loss: 0.2225
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 88s 126ms/step - accuracy: 0.9799 - loss: 0.0596 - val_accuracy: 0.9533 - val_loss: 0.1531
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 117ms/step - accuracy: 0.9850 - loss:

In [ ]:
import cv2
import matplotlib as plt
test_img = cv2.imread('content/dog.jpg')
plt.imshow(test_img)

AttributeError: module 'matplotlib' has no attribute 'imshow'

In [ ]:
test_img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
test_img = cv2.resize(test_img,(256,256))
test_input = test_img.reshap((1,256,256,3))
model.predict(test_input)